In [1]:
import pandas as pd
import regex as re
from sklearn.utils import resample
import warnings 
warnings.filterwarnings('ignore')
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
import re 

# some insights of data 
text given in each datapoint is a tweet or grabage values with is in many languages.
the data is given in 4 files with label as the file name.
as machine can be trained by only numerical values , we have to convert it to vectors ,before that all the text should be in same language ,
so we can translate the text from non-english to english and train the machine or by dropping tweets which are in non-english language


In [2]:
litigious_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\litigious.csv",encoding='latin-1')
negative_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\negative.csv",encoding='latin-1')
positive_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\positive.csv",encoding='latin-1')
uncertainty_data = pd.read_csv(r"C:\Users\Home\Downloads\assignmet 2\uncertainty.csv",encoding='latin-1')

In [3]:
litigious_data['new_column']='litigious'
negative_data['new_column']='negative'
positive_data['new_column']='positive'
uncertainty_data['new_column']='uncertainty'

As the label is given as name of csv file i have added a column with column name as new_column for every dataframe

In [21]:
data = pd.concat([litigious_data,negative_data,positive_data,uncertainty_data])

concatinating all the 4 dataframes in to one 

In [22]:
data

,Unnamed: 0,Text,Language,new_column
0,0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,litigious
1,2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,litigious
2,4,OOPS. I typed her name incorrectly (todayâs ...,en,litigious
3,14,@SaltBurned [When the first sign of surrender ...,en,litigious
4,27,"""It's Your fight to save the UK from the liars...",en,litigious
...,...,...,...,...
206935,937827,There is a bag of cheese hidden somewhere in m...,en,uncertainty
206936,937833,@Treg2Cole Thatâs a hellz nah for me. ð¤¢. ...,en,uncertainty
206937,937834,@Sam_Mallory El Mizouni and Roles in the same ...,en,uncertainty
206938,937838,If you donât take risks youâll always end ...,en,uncertainty


In [23]:
data = data.reset_index()

reseting all the index values as we have concatinated many dataframes to one 

In [24]:
data.drop(labels=['index','Unnamed: 0'],axis=1,inplace=True)

removing unwanted columns

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 937854 entries, 0 to 937853
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Text        937854 non-null  object
 1   Language    937831 non-null  object
 2   new_column  937854 non-null  object
dtypes: object(3)
memory usage: 21.5+ MB


In [26]:
data['Language'].value_counts()

en                                                                                                                                                                      871310
fr                                                                                                                                                                       13091
es                                                                                                                                                                       11333
pt                                                                                                                                                                       10336
ja                                                                                                                                                                        8414
                                                                                                                             

As the majority of the language tweets are of english (~93%),it takes much time to translate non-english tweet to english, we can remove remaining rows which are in other language

In [28]:
data = data[data['Language']=='en']

only enlgish language tweets are present in the dataframe so we can do conversion to vector easily 

In [29]:
data

,Text,Language,new_column
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,litigious
1,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,litigious
2,OOPS. I typed her name incorrectly (todayâs ...,en,litigious
3,@SaltBurned [When the first sign of surrender ...,en,litigious
4,"""It's Your fight to save the UK from the liars...",en,litigious
...,...,...,...
937849,There is a bag of cheese hidden somewhere in m...,en,uncertainty
937850,@Treg2Cole Thatâs a hellz nah for me. ð¤¢. ...,en,uncertainty
937851,@Sam_Mallory El Mizouni and Roles in the same ...,en,uncertainty
937852,If you donât take risks youâll always end ...,en,uncertainty


In [30]:
data.isnull().sum()

Text          0
Language      0
new_column    0
dtype: int64

In [31]:
stop_words = stopwords.words("english")
stop_words.remove('not')

this stop words which occur often and meaning of sentance will not be changed if they are not present , so that dimensions of the vector can be reduced , not is removed assuming that if that is removed from the sentance the meaning could be reversed  

In [39]:
%%time
sentances = []
for i in data['Text']:
    word = []
    sent = i.lower()
    sent = emoji.demojize(sent)
    sent = re.sub('â€˜|â€™',"'",sent)
    sent = re.sub('&amp;',"and",sent)
    sent = re.sub('<.*?>','',sent)
    sent = re.sub('https?://\S*|www\S*','',sent)
    sent = re.sub('@\w+',"",sent)
    sent = re.sub('#\w+',"",sent)
    sent = re.sub("[^a-z0-9' ]","",sent)
    for i in word_tokenize(sent):
        if i in stop_words:pass
        else:word.append(PorterStemmer().stem(i))
    sentances.append(" ".join(word))

Wall time: 10min 49s


time taken to do porterstemmer on 871310 rows is 13min 6sec, in this step we have done text pre-processing , this all the steps inside are done for redcution of vector dimensions


In [40]:
cleaned_text_data = pd.DataFrame({"cleaned_Text":sentances})

In [41]:
cleaned_text_data

,cleaned_Text
0,testimoni not evid court law state feder must ...
1,flagstar bank disclos data breach impact 15mil...
2,oop type name incorrectli today brave wit 6 7 ...
3,first sign surrend come michael smile split fa...
4,'s fight save uk liar cheat charlatan steve br...
...,...
871305,bag chees hidden somewher hous aint talk
871306,that hellz nah mayb fri
871307,el mizouni role team could realli good colin p...
871308,dont take risk youll alway end work someon kin...


creating dataframe of each data piont in text column 

In [42]:
#bag of words
count_obj = CountVectorizer()
trans = count_obj.fit_transform(cleaned_text_data['cleaned_Text'][0:25000])
trans

<25000x31406 sparse matrix of type '<class 'numpy.int64'>'
	with 333416 stored elements in Compressed Sparse Row format>

there are to 333416 unique words 

In [43]:
array_count = trans.toarray()

In [56]:
count_obj.vocabulary_

{'testimoni': 27611,
 'not': 19726,
 'evid': 10535,
 'court': 7698,
 'law': 16346,
 'state': 26489,
 'feder': 11069,
 'must': 18967,
 'stand': 26430,
 'cross': 7953,
 'examin': 10578,
 'flagstar': 11348,
 'bank': 4095,
 'disclos': 9075,
 'data': 8268,
 'breach': 5131,
 'impact': 14266,
 '15million': 418,
 'individu': 14468,
 'oop': 20342,
 'type': 28888,
 'name': 19076,
 'incorrectli': 14408,
 'today': 28171,
 'brave': 5112,
 'wit': 30618,
 'probabi': 22256,
 'dont': 9378,
 'forget': 11557,
 'request': 23575,
 'first': 11298,
 'sign': 25549,
 'surrend': 27045,
 'come': 6915,
 'michael': 18171,
 'smile': 25872,
 'split': 26278,
 'face': 10809,
 'like': 16812,
 'sharp': 25299,
 'knife': 15972,
 'flesh': 11376,
 'would': 30876,
 'chill': 6357,
 'dean': 8386,
 'core': 7538,
 'ice': 14079,
 'cold': 6841,
 'exist': 10655,
 'natur': 19149,
 'archangel': 3380,
 'enj': 10161,
 'fight': 11202,
 'save': 24581,
 'uk': 28956,
 'liar': 16716,
 'cheat': 6261,
 'charlatan': 6232,
 'steve': 26594,
 'br

this command gives all the unique words with there index value in the vector 

In [52]:
pd.DataFrame(array_count,columns=sorted(count_obj.vocabulary_))

,00,000,000001,0007,001,0047,004coinmarketcap,006,007,007anoth,...,zuckerbergfund,zuckerbrod,zuhu,zuma,zumwalt,zuneth,zygotes3,zylstra,zzzz,zzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
#TF-IDF
tf_obj = TfidfVectorizer()
trans_tf = tf_obj.fit_transform(cleaned_text_data['cleaned_Text'][0:25000])
trans_tf

<25000x31406 sparse matrix of type '<class 'numpy.float64'>'
	with 333416 stored elements in Compressed Sparse Row format>

In [58]:
tf_obj.vocabulary_

{'testimoni': 27611,
 'not': 19726,
 'evid': 10535,
 'court': 7698,
 'law': 16346,
 'state': 26489,
 'feder': 11069,
 'must': 18967,
 'stand': 26430,
 'cross': 7953,
 'examin': 10578,
 'flagstar': 11348,
 'bank': 4095,
 'disclos': 9075,
 'data': 8268,
 'breach': 5131,
 'impact': 14266,
 '15million': 418,
 'individu': 14468,
 'oop': 20342,
 'type': 28888,
 'name': 19076,
 'incorrectli': 14408,
 'today': 28171,
 'brave': 5112,
 'wit': 30618,
 'probabi': 22256,
 'dont': 9378,
 'forget': 11557,
 'request': 23575,
 'first': 11298,
 'sign': 25549,
 'surrend': 27045,
 'come': 6915,
 'michael': 18171,
 'smile': 25872,
 'split': 26278,
 'face': 10809,
 'like': 16812,
 'sharp': 25299,
 'knife': 15972,
 'flesh': 11376,
 'would': 30876,
 'chill': 6357,
 'dean': 8386,
 'core': 7538,
 'ice': 14079,
 'cold': 6841,
 'exist': 10655,
 'natur': 19149,
 'archangel': 3380,
 'enj': 10161,
 'fight': 11202,
 'save': 24581,
 'uk': 28956,
 'liar': 16716,
 'cheat': 6261,
 'charlatan': 6232,
 'steve': 26594,
 'br

In [49]:
pd.DataFrame(trans_tf.toarray(),columns=sorted(tf_obj.vocabulary_.keys()))

,00,000,000001,0007,001,0047,004coinmarketcap,006,007,007anoth,...,zuckerbergfund,zuckerbrod,zuhu,zuma,zumwalt,zuneth,zygotes3,zylstra,zzzz,zzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


in countvectorizer (bag of words) and tfidfvectorizer(TF-IDF) i have reduced datapoints because of memory problem so i have limited to 25000 rows in countvectorizer instead of 871310 and 5000 in tfidfvectorizer 